# Introduction

This notebook illustrates clustering of Ti crystal misorientations using data obtained from a highly deformed specimen, using EBSD.

This functionaility has been checked to run in orix-0.2.3 (May 2020). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/orix-demos/issues

# Contents

1. <a href='#imp'> Import data</a>
2. <a href='#dis'> Compute distance matrix</a>
3. <a href='#clu'> Clustering</a>
4. <a href='#vis'> Visualisation</a>

Import orix classes and various dependencies

In [ ]:
%matplotlib qt5

# Important external dependencies
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# Import orix classes and functions
from orix.quaternion.orientation import Orientation, Misorientation
from orix.quaternion.symmetry import D6
from orix.quaternion.orientation_region import OrientationRegion
from orix.vector import Vector3d
from orix.quaternion.rotation import Rotation
from orix.vector.neo_euler import AxAngle
from orix import plot

# Colorisation
from skimage.color import label2rgb
from matplotlib.colors import to_rgb, to_hex

# Visualisation
from matplotlib.lines import Line2D

# <a id='imp'></a> 1. Import data

Load orientation mapping data specified in a standard CTF file as euler angles in degrees, following the Bunge convention.

In [ ]:
filepath = './data/Ti_orientations.ctf'
dat = np.loadtxt(filepath, skiprows=1)[:, :3]

Initialize an orix Orientation object containing the data, with reshape, subset selection and symmetry setting

In [ ]:
ori = Orientation.from_euler(np.radians(dat), convention="Krakow_Hielscher").reshape(381, 507)[-100:, :200].set_symmetry(D6)

Compute misorientations and set symmetry

In [ ]:
misori_base = Misorientation(~ori[:, :-1] * ori[:, 1:])
boundary_mask = misori_base.angle > np.radians(7)
misori = misori_base[boundary_mask].set_symmetry(D6, D6)
print('Number of misorientations:', misori.size)

Define the fundamental region based on the D6 symmetry of Ti

In [ ]:
fundamental_region = OrientationRegion.from_symmetry(D6, D6)

# <a id='dis'></a> 2. Compute distance matrix

Clustering algorithms require a distance matrix, $D_{ij}$, containing the distance, $d(o_i, o_j)$, between all (mis)orientations to be computed. We define this distance as the minimum rotational angle relating (mis)orientations amongst all symmetry equivalent rotations.

Computation of the distance matrix is the most computationally intensive part of this data processing. Here we provide 3 alternative implementations that use resources differently:

1. Calculate the outer products needed to determine the distance metric and compute the associated angle. Minimise with respect to the tensor axes corresponding to symmetry. Uses lots of of computer memory.


2. Iterating over pairs of data points while performing an outer product minimisation with respect to the symmetry elements for each pair. Uses more computer memory to avoid excessive computation times.


**WARNING: The computation in Section 2.1 takes time and may exceed limits on your machine. This section is commented out to avoid unintentional use. A pre-computed solution is provided in Section 2.2!**

## 2.1. Compute the distance matrix yourself (optional)

**Option 1: high (ca. 32 Gb) RAM, fast vectorized computation**

Computes every possibility in a single tensor, then minimises.

In [ ]:
#mismisori = (~misori).outer(misori)
#mismisori_equiv = D6.outer(~misori).outer(D6).outer(D6).outer(misori).outer(D6)
#distance = mismisori_equiv.angle.data.min(axis=(0, 2, 3, 5))

**Option 3: RAM vs. speed compromise**

Precomputes one set of equivalent orientations.

In [ ]:
from itertools import combinations_with_replacement as icombinations
from tqdm import tqdm_notebook
distance = np.empty((misori.size, misori.size))

for i, j in tqdm_notebook(list(icombinations(range(misori.size), 2))):
    m_1, m_2 = misori[i], misori[j]
    mismisori = D6.outer(~m_1).outer(D6).outer(D6).outer(m_2).outer(D6)
    d = mismisori.angle.data.min(axis=(0, 2, 3, 5))
    distance[i, j] = d
    distance[j, i] = d

## 2.2. Load a distance matrix we computed for you

In [ ]:
filepath_2 = './data/misori-distance((100, 200)).npy'
distance = np.load(filepath_2)

# <a id='clu'></a> 3. Clustering

Apply mask to remove small misorientations associated with grain orientation spread

In [ ]:
small_mask = misori.angle < np.radians(7)
distance = distance[~small_mask][:, ~small_mask]
misori = misori[~small_mask]

Perform clustering

In [ ]:
# Compute clusters
dbscan = DBSCAN(0.05, 10, metric='precomputed').fit(distance)
print('Cluster labels:', np.unique(dbscan.labels_))
n_clusters = len(np.unique(dbscan.labels_)) - 1
print('Number of clusters:', n_clusters)

Calculate the mean misorientation associated with each cluster

In [ ]:
theta = 15
rc = Orientation([np.cos(np.deg2rad(theta/2)),0,0,np.sin(np.deg2rad(theta/2))])
list_of_means = []
for label in np.unique(dbscan.labels_)[1:]:
    cluster_mean_local = (~rc) * ((rc * misori[dbscan.labels_ == label]).set_symmetry(D6,D6).mean())
    list_of_means.append(cluster_mean_local)
cluster_means = Misorientation.stack(list_of_means).flatten()
cluster_means = cluster_means.set_symmetry(D6, D6)

Inspect cluster means in axis-angle representation

In [ ]:
np.degrees(cluster_means.angle.data)

In [ ]:
cluster_means.axis

Define reference misorientations associated with twinning orientation relationships

In [ ]:
# From Krakow et al
sigma7a = Rotation.from_neo_euler(AxAngle.from_axes_angles((1, 0, 0), np.radians(64.40)))
sigma11a = Rotation.from_neo_euler(AxAngle.from_axes_angles((1, 0, 0), np.radians(34.96)))
sigma11b = Rotation.from_neo_euler(AxAngle.from_axes_angles((2, 1, 0), np.radians(85.03)))
sigma13a = Rotation.from_neo_euler(AxAngle.from_axes_angles((1, 0, 0), np.radians(76.89)))
sigma13b = Rotation.from_neo_euler(AxAngle.from_axes_angles((2, 1, 0), np.radians(57.22)))

twin_theory = Orientation.stack([sigma7a, sigma11a, sigma11b, sigma13a, sigma13b]).flatten()

Calculate difference, defined as minimum rotation angle, between measured and theoretical values

In [ ]:
mismisori = (~twin_theory).outer(cluster_means)
mismisori_equiv = D6.outer(~twin_theory).outer(D6).outer(D6).outer(cluster_means).outer(D6)
distance = mismisori_equiv.angle.data.min(axis=(0, 2, 3, 5))

In [ ]:
np.degrees(distance)

# <a id='vis'></a> 4. Visualisation

Associate colors with clusters for plotting

In [ ]:
colors = [to_rgb('C{}'.format(i)) for i in range(10)]
c = label2rgb(dbscan.labels_, colors=colors)

Plot the misorientation clusters within the fundamental zone for D6, D6 bicrystal symmetry

In [ ]:
fig = plt.figure(figsize=(3.484252, 3.484252))
gridspec = plt.GridSpec(1, 1, left=0, right=1, bottom=0, top=1, hspace=0.05)


ax_misori = fig.add_subplot(gridspec[0], projection='axangle', aspect='equal', proj_type='ortho')
ax_misori.scatter(misori,s=4,c=c) 
ax_misori.plot_wireframe(fundamental_region, color='black', linewidth=0.5, alpha=0.1, rcount=361, ccount=361)


ax_misori.set_axis_off()
ax_misori.set_xlim(0.2, 1.2)
ax_misori.set_ylim(-.1, .9)
ax_misori.set_zlim(-0, 1)
ax_misori.view_init(90, -60)


handles = [
    Line2D(
        [0], [0], 
        marker='o', color='none', 
        label=i+1, 
        markerfacecolor=color, markersize=5
    ) for i, color in enumerate(colors[:n_clusters])
]

ax_misori.legend(handles=handles, loc='upper left')

Plot side view of misorientation clusters in the fundamental zone for D6, D6 bicrystal symmetry

In [ ]:
fig = plt.figure(figsize=(3.484252*2, 1.5*2))
gridspec = plt.GridSpec(1, 1, left=0, right=1, bottom=0, top=1, hspace=0.05)

ax_misori = fig.add_subplot(gridspec[0], projection='axangle', proj_type='ortho', aspect='equal')
ax_misori.scatter(misori, c=c, s=4)
ax_misori.plot_wireframe(fundamental_region, color='black', linewidth=0.5, alpha=0.1, rcount=181, ccount=361)

ax_misori.set_axis_off()
ax_misori.set_xlim(0.1, 1.1)
ax_misori.set_ylim(0.1, 1.1)
ax_misori.set_zlim(-0, 1)
ax_misori.view_init(0, -60)

Generate map of boundaries colored according to cluster membership

In [ ]:
mapping = np.ones(misori_base.shape + (3,))
mapping[np.where(boundary_mask)[0][~small_mask], np.where(boundary_mask)[1][~small_mask]] = c

Plot map of boundaries colored according to cluster membership

In [ ]:
fig = plt.figure(figsize=(3.484252, 2))

gridspec = plt.GridSpec(1, 1, left=0, right=1, bottom=0, top=1, hspace=0.05)
ax_mapping = fig.add_subplot(gridspec[0])
ax_mapping.imshow(mapping)

ax_mapping.set_xticks([])
ax_mapping.set_yticks([])